In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("netflix_titles.csv")

In [3]:
data ['tags'] = data['director'] + ' ' + data['cast']+ ' ' + data['listed_in'] + ' ' + data['description']

In [4]:
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,tags
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",NaN
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",NaN
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,"Julien Leclercq Sami Bouajila, Tracy Gotoas, S..."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",NaN
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,NaN


In [5]:
data = data[['show_id','type','title', 'tags']]

In [6]:
#drop the columns that has NA value in their tags
data.dropna(inplace=True)

In [7]:
data.reset_index(inplace =True, drop=True)

In [8]:
data

,show_id,type,title,tags
0,s3,TV Show,Ganglands,"Julien Leclercq Sami Bouajila, Tracy Gotoas, S..."
1,s6,TV Show,Midnight Mass,"Mike Flanagan Kate Siegel, Zach Gilford, Hamis..."
2,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha Vanessa Hudgens,..."
3,s8,Movie,Sankofa,"Haile Gerima Kofi Ghanaba, Oyafunmike Ogunlano..."
4,s9,TV Show,The Great British Baking Show,"Andy Devonshire Mel Giedroyc, Sue Perkins, Mar..."
...,...,...,...,...
5695,s8802,Movie,Zinzana,"Majid Al Ansari Ali Suliman, Saleh Bakri, Yasa..."
5696,s8803,Movie,Zodiac,"David Fincher Mark Ruffalo, Jake Gyllenhaal, R..."
5697,s8805,Movie,Zombieland,"Ruben Fleischer Jesse Eisenberg, Woody Harrels..."
5698,s8806,Movie,Zoom,"Peter Hewitt Tim Allen, Courteney Cox, Chevy C..."


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
cv = CountVectorizer(max_features=5700,stop_words= 'english')

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
vector = cv.fit_transform(data['tags'].values.astype('U')).toarray()

In [23]:
similarity = cosine_similarity(vector)

In [24]:
similarity

array([[1.        , 0.21764288, 0.05647825, ..., 0.        , 0.03121953,
        0.06024752],
       [0.21764288, 1.        , 0.02594996, ..., 0.02923527, 0.05737753,
        0.02768183],
       [0.05647825, 0.02594996, 1.        , ..., 0.03413944, 0.13400504,
        0.09697623],
       ...,
       [0.        , 0.02923527, 0.03413944, ..., 1.        , 0.1132277 ,
        0.03641785],
       [0.03121953, 0.05737753, 0.13400504, ..., 0.1132277 , 1.        ,
        0.07147417],
       [0.06024752, 0.02768183, 0.09697623, ..., 0.03641785, 0.07147417,
        1.        ]])

In [25]:
data[data['title']=='Zodiac'].index[0]

5696

In [27]:
similar_movies = sorted(list(enumerate(similarity[5696])),reverse=True, key=lambda vector:vector[1])

In [28]:
for i in similar_movies[1:6]:
    print(data.iloc[i[0]].title)
    print(i[1])

To and From New York
0.2637521893583148
Gothika
0.2637521893583148
WHAT DID JACK DO?
0.2449489742783178
Interrogation
0.223606797749979
The Highwaymen
0.22019275302527214


In [38]:
def recommend_movie(movie_name):
    movie_index = data[data['title'] == movie_name].index[0]
    similar_movies = sorted(list(enumerate(similarity[movie_index])),reverse=True, key=lambda vector:vector[1])
    for i in similar_movies[1:6]:
        print(data.iloc[i[0]].title +' with similarity of ' + str(round(i[1]*100,2)))

In [39]:
recommend_movie('Ganglands')

Bangkok Breaking with similarity of 51.21
Dealer with similarity of 50.83
The Forest of Love: Deep Cut with similarity of 50.74
Sleepless Society: Two Pillows & A Lost Soul with similarity of 49.67
Unlucky Ploy with similarity of 47.36


In [40]:
import pickle as pk

In [41]:
pk.dump(data, open('movie_dataset.pkl','wb'))

In [42]:
pk.dump(similarity, open('similarity.pkl','wb'))